In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Heading
After having gone through first dataset at (put link in future) this new dataset actually tells us the model. It also has robust features to make better ML models.

In [4]:
def df_splits(df, col, val='Yes', strat='Yes', seed=42):
    '''
    This function takes in a dataframe and a taget column, as well as several optional arguments. You can decide if you want a validate set and if you want to stratify. 
    By leaving those two variables alone you will stratify on the target column and get a validate subset. If you change them to anything when calling the function,
    you will not get a validate or stratification. There is also an argument for the seed, which is set to 42 by default.
    '''
    # If val is left alone at 'Yes', the function will run this loop and return a validate subset.
    if val == 'Yes':
        # If strat is left alone at 'Yes', the function will stratify by the column named when calling the function.
        if strat == 'Yes':
            # Train, validate, and test subsets created.
            train, val_test = train_test_split(df, train_size=.6, random_state=seed, stratify=df[col])
            validate, test = train_test_split(val_test, train_size=.6, random_state=seed, stratify=val_test[col])
            # Printing the shapes of each subset 
            print(train.shape, validate.shape, test.shape)
            return train, validate, test
        # If the strat argument is changed at all, it will default to not doing it and not stratify when splitting.
        else:
            #Splitting the data into train, validate, and test.
            train, val_test = train_test_split(df, train_size=.6, random_state=seed)
            validate, test = train_test_split(val_test, train_size=.6, random_state=seed)
            # Again, printing the subset
            print(train.shape, validate.shape, test.shape)
            return train, validate, test
    # This part of the loop is for if you changed the val argument to something other than 'Yes', which will make it not create a validate subset. 
    else:
        # If strat is left at 'Yes', the function will stratify on the column named when calling the function
        if strat == 'Yes':
            # Splitting the data into train and test subsets
            train, test = train_test_split(df, train_size=.8, random_state=seed, stratify=df[col])
            # Printing the shapes
            print(train.shape, test.shape)
            return train, test

        else:
            # Splitting the data into train and test
            train, test = train_test_split(df, train_size=.8, random_state=seed)
            # Printing the shapes
            print(train.shape, test.shape)
            return train, test

The above function splits data and gives stratifying options. From here we import the data and bring it in

In [14]:
df = pd.read_csv('Cleaned_Laptop_data.csv')
df.columns

Index(['brand', 'model', 'processor_brand', 'processor_name',
       'processor_gnrtn', 'ram_gb', 'ram_type', 'ssd', 'hdd', 'os', 'os_bit',
       'graphic_card_gb', 'weight', 'display_size', 'warranty', 'Touchscreen',
       'msoffice', 'latest_price', 'old_price', 'discount', 'star_rating',
       'ratings', 'reviews'],
      dtype='object')

Adding lastest_price and old_price and dividng to get a working average
- working_average
# Now going into Column to column
We are assembling our questions for this dataset
I am starting at the bottom

In [19]:
df['reviews'].nunique()

152

The above finding is bizarre, I think this may be a mix of rating expressions

In [24]:
pd.DataFrame(df['reviews']).head(5)
#oops this NUMBER of reviews, not the reviews itself
#Can possibly clean up for clarity sake

,reviews
0,1947
1,108
2,4
3,18
4,15


# The reviews column
It is the number of reviews and maybe useful 

In [26]:
pd.DataFrame(df['ratings']).head(5)
#number of ratings

,ratings
0,15279
1,990
2,28
3,158
4,116


In [29]:
pd.DataFrame(df['star_rating']).head(5)
#THIS is what I was looking for a universal metric to for reviewing

,star_rating
0,3.8
1,4.3
2,3.9
3,4.4
4,4.2


In [31]:
df.star_rating.min()
#at position lowest star_rating review so like floor or min

0.0

In [33]:
df[df['star_rating'] == 0] 
#in the new findings there are a sizeable amount of 0s in the dataset for ratings which is worth interrogating

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
42,Lenovo,Core,Intel,Core i5,11th,8,DDR4,512,0,Windows,...,15.6,1,No,No,52990,84890,37,0.0,0,0
43,acer,Aspire,Intel,Core i3,11th,8,DDR4,256,0,Windows,...,15.6,1,No,No,38990,52999,26,0.0,0,0
77,Lenovo,IdeaPad,Intel,Core i5,10th,8,DDR4,512,0,Windows,...,15.6,1,No,No,49990,79690,37,0.0,0,0
90,ASUS,VivoBook,Intel,Core i5,11th,8,DDR4,256,1024,Windows,...,15.6,1,No,No,52990,72990,27,0.0,0,0
99,ASUS,Zenbook,Intel,Core i7,11th,16,LPDDR4X,1024,0,Windows,...,14,1,Yes,Yes,104990,146990,28,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0
876,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0
877,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0
878,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0


In [ ]:
train, validate, test = df_splits(df, "")